In [ ]:
import math
from IPython.display import clear_output
from itertools import chain
import numpy as np
import time

#players
player_1 = 'Black'
player_2 = 'White'

                                        #3.1. Choose a formalism to represent all states: 15%
                                        
                                        #1. Array of possible states A1 - E5:
                                        #(columns = 1-5, rows = A-E)
                                        #2. Dictionary of possible connections
                                        #3. Array of possible mills
#1. Array of possible states                                    
#board as a 1Dimensional list
board = ['A1','A3', 'A5', 
         'B2','B3', 'B4', 
       'C1', 'C2', 'C4','C5', 
         'D2', 'D3','D4', 
         'E1', 'E3','E5']

#curent state set as board
cur = board.copy() 

#2. Dictionary of possible connections, Key = point, Values = connecting points
connections = {'A1': ['A3', 'C1'], 
               'A3': ['A1', 'A5', 'B3'], 
               'A5': ['A3', 'C5'], 
               
               'B2': ['B3', 'C2'], 
               'B3': ['B4', 'B2', 'A3'],
               'B4': ['B3', 'C4'], 
               
               'C1': ['C2', 'A1', 'E1'],
               'C2': ['B2', 'D2', 'C1'], 
               'C4': ['C5', 'B4', 'D4'],
               'C5': ['C4', 'A5', 'E5'], 
               
               'D4': ['C4', 'D3'], 
               'D2': ['C2', 'D3'], 
               'D3': ['D2', 'D4'], 
               
               'E1': ['C1', 'E3'], 
               'E3': ['E1', 'D3', 'E5'], 
               'E5': ['E3', 'C5']}

#3. 2D array of possible mills
mills = [['A1', 'A3', 'A5'], ['A1', 'C1', 'E1'], ['A5', 'C5', 'E5'], ['E1', 'E3', 'E5'], ['B2', 'B3', 'B4'],
        ['B2', 'C2', 'D2'], ['B4', 'C4', 'C4'], ['D2', 'D3', 'D4']]

def possible_moves(player, point): #this function will find possible moves,
                                    #given a player and a point on the board
    possible_moves = []
    for t in connections:
        if cur[board.index(point)] == player:
            if point == t:
                for r in (connections[t]):
                    if (cur[board.index(r)]) != player_1 and (cur[board.index(r)]) != player_2:
                        possible_moves.append(r)
    return possible_moves  







            
                
                                        #3.2. Choose an Evaluation Function for a non-terminal state: 10%
                
def evalState(cur, player):
    #print('evalState', player)         #This function will evaluate a non-terminal state for a player.
    total = 0                           #evalState finds the difference between player's and opponents no.s of men:
    winning = 0                         #the greater the +ve value, the more the player is winning,
    losing = 0                          #the greater the -ve value, the more the player is losing
    if player == player_1:
        for i in cur: 
            if i == player_1:
                winning += 1
            if i == player_2: 
                losing += 1
        total = winning - losing
    return total
    if player == player_2:
        for i in cur: 
            if i == player_2:
                winning += 1
            if i == player_1: 
                losing += 1
        total = winning - losing
    
    return total
    
def winnerfound(state): #this function will determine if a player has won the game

    player1count = 0
    player2count = 0
    for i in state:
        if i == player_1:
            player1count += 1
        if i == player_2:
            player2count += 1
    if player1count < 3:  #player loses when reduced to less than three players
        print('winner = player 2')
        return True
    if player2count < 3:
        print('winner = player 1')
        return True
    return False

def placing_pieces_human(state): #this function will determine if part1 of gameplay is finished for human

    player2count = 0
    for i in range(len(state)):
        if state[i] == player_2:
            player2count +=1
    if player2count == 6: 
        return True
    return False

def placing_pieces_computer(state): #this function will determine if part1 of gameplay is finished for AI

    player1count = 0
    for i in state:
        if i == player_1:
            player1count +=1
    if player1count == 6:  
        return True
    return False
                                            
    
    
    
    
    
    
                                        #3.3. Implement the AlphaBeta Algorithm with a depth of 8: 25%
                        
def MinimaxDecision(cur): #1. Minimax Decision selects action for Part 1 which maximises output
    
  # check the Max value for all possible actions
  maxVal = -20
  action = 0
  for i in range(len(board)):
    if (cur[i] != player_1 and cur[i] != player_2): # it is a possible action
    #create the nextState and send it to MinValue
        nextState = [cur[ti] for ti in range(len(board))]
        nextState[i] = player_1
        stealmill(i, player_1, nextState)
        val = MinValue(nextState, -math.inf, math.inf, 8) #depth of 8
        if (val > maxVal):
          maxVal = val
          action = i
  # return the action that maximizes the output
  return action

def MinimaxDecision_stealingmills(cur, player, depth): #2. Selects opponent's piece to remove
                                                            #which maximises output
  if (depth == 0): 
    return None, evalState(cur, player_1)
  # check the Max value for all possible actions
  maxVal = 20
  if player == player_1: 
        maxVal = -20

  action = 0
  for i in range(len(board)):
    if cur[i] != player and cur[i] != board[i] : # for all possible opponents to remove
    #create the nextState and send it to MinValue
        nextState = [cur[ti] for ti in range(len(board))]
        nextState[i] = board[i]
        if player == player_1: 
            val = MinValue(nextState, -math.inf, math.inf, depth-1)
            
            if (val > maxVal):
              maxVal = val
              action = i
        if player == player_2: 
            
            val = MaxValue(nextState, -math.inf, math.inf, depth-1)
            #print(val)
            if (val < maxVal):
              maxVal = val
              action = i
  # return the action that maximizes the output
  return action, maxVal, 

def MinimaxDecision_movingplayer(cur): #3. MinimaxDecision_movingplayer moves a player, 
                                       #given possible moves and players on board, 
                                       #to maximise output

  # check the Max value for all possible actions
  maxVal = -20
  actions = []
  for i in range(len(board)):
    if (cur[i] == player_1): #for all Men
        for t in range(len(board)):
            if len(possible_moves(player_1, board[i])) > 0: #if there are possible moves with this player
                for element in possible_moves(player_1, board[i]): 
    #create the nextState and send it to MinValue
                    nextState = cur
                    nextState[t] = player_1
                    nextState[i] = board[i]
                    val = MinValue(nextState, -math.inf, math.inf, 8) #depth of 8
                    if (val > maxVal):
                        maxVal = val
                        action_move = t
                        action_leave = i
                        actions.append(action_move)
                        actions.append(action_leave) 
  # return the action that maximizes the output

  return actions


def MaxValue(cur, alpha, beta, depth): #4. MaxValue - calculates the maximum value of board
    
  if (depth == 0):
    #print('Test')
    return evalState(cur, player_1)
    
  # check the Max value for all possible actions
  maxVal = -20
  for i in range(len(board)):
    if (cur[i] != player_1 and cur[i] != player_2): # it is a possible action
        #create the nextState and send it to MinValue
        nextState = [cur[x] for x in range(len(board))]
        nextState[i] = player_1
        
        if stealmill(i, player_1, nextState):
            val = MinimaxDecision_stealingmills(cur, player_1, depth-1)[1]
        else:
            val = MinValue(nextState, alpha, beta, depth-1)
        
        if (val > maxVal):
            maxVal = val
        if (val >= beta):
            return val
        if (val > alpha):
            alpha = val
  return maxVal

def MinValue(cur, alpha, beta, depth): #5. MinValue - calculates the minimum value of board
    
  if (depth == 0):
    #print('Test')
    return evalState(cur, player_1)
  # check the Max value for all possible actions
  minVal = 20
  for i in range(len(board)):
    if (cur[i] != player_1 and cur[i] != player_2): # it is a possible action
        #create the nextState and send it to MaxValue
        nextState = [cur[x] for x in range(len(board))]
        nextState[i] = player_2
        if stealmill(i, player_1, nextState):
            val = MinimaxDecision_stealingmills(cur, player_2, depth-1)[1]
        else:
            val = MaxValue(nextState, alpha, beta, depth-1)
        if (val < minVal):
          minVal = val
        if (val <= alpha):
          return val
        if (val < beta):
          beta = val
  return minVal
    
    
    
    
    
    
    
    
                                        #3.4. Implement the Main function (AI vs Human) 10%
def game_play():   
    def part_1():                       #part 1 = player places pieces on the board 
        
        printGameBoard(cur)
        def human_player(player): #human player
            
            validR = False
            while validR == False:
                R = str(input('Which point do you want to place a piece on?: ' ))
                if R not in board: 
                    return('Try again')
                if cur[board.index(R)] == 'Black' or cur[board.index(R)] == 'White':
                    return('Thats already taken! Pick another...')
                else:
                    validR = True
                    cur[board.index(R)] = player

                    printGameBoard(cur)
                    domill(board.index(R), player, cur) #try to find and steal mills




        def AI_player(player): #AI player (using AlphhaBeta + Minimax algorithms)

            action = MinimaxDecision(cur)

            cur[action] = player #implement the action found by Minimax Decision function
            printGameBoard(cur)
            domill(action, player, cur)

        
        

        while placing_pieces_human(cur) == False or placing_pieces_computer(cur) == False:
            if placing_pieces_human(cur) == False:
                human_player(player_2) #human player
            if placing_pieces_computer(cur) == False: 
                AI_player(player_1)    #AI player        

            
    
    
    def part_2():                            #Part2 = Moving pieces, and removing opponent
                     
                                             #player will select piece on board,
                                             #be given options for possible moves,
                                             #and can move that piece there
        print('You have reached part 2...')
        def human_player_2(player):
            
            
                     
            selected_player = False
            while selected_player == False:
            
                moving_player = str(input("You may move one player. Pick your player: " ))
                if cur[board.index(moving_player)] == player:
                     #if player has chosen one of their Men to move...
                    
                    for moves in possible_moves(player, moving_player): 
                        if cur[board.index(moves)] != player_1 and cur[board.index(moves)] != player_2: 
                        
                            selected_player = True
                    
            possiblemoves = possible_moves(player, moving_player)
            for move in possiblemoves: 
                if cur[board.index(move)] == player_1 or cur[board.index(move)] == player_2: 
                    continue
                else:
                    print(move)
                
            moving_to = str(input("Where will you move your player: " ))
                    #player can select from series of options generated by possible_moves function
            
            if moving_to in possiblemoves:
                
                    #player's piece will move
                cur[board.index(moving_to)] = player
                cur[board.index(moving_player)] = moving_player
            
            printGameBoard(cur)
            domill(board.index(moving_to), player, cur) #stealmill function, so player can continue to remove opponent's men
            printGameBoard(cur)
            
        def AI_player_2(player): #AI player (using AlphhaBeta + Minimax algorithms)
            
            actions = MinimaxDecision_movingplayer(cur)
            #print(actions)
            move_to = actions[0]
            move_from = actions[1]
            cur[move_to] = player
            cur[move_from] = board[move_from]
            
            printGameBoard(cur)
            domill(move_to, player, cur)              #stealmill function, so player can continue to remove opponent's men
            
        

        while winnerfound(cur) == False:
 
            human_player_2(player_2)
            AI_player_2(player_1)
        

    part_1() 
    part_2()
    
    
def stealmill(new_index, player, state):  
    
                                                #this function will find whether a mill has been formed,
                                                #and remove an opponents player.
    
    mill = False
    for t in mills: 
            
        # if the last move played by player completed that mill (so not 2,3,4 moves back, only the last move counts)
        if new_index != board.index(t[0]) and new_index != board.index(t[1]) and new_index != board.index(t[2]):
            continue
            
        #...and all parts of the mill belong to the same player (i.e. are equal)
        if state[board.index(t[0])] != state[board.index(t[1])] or state[board.index(t[1])] != state[board.index(t[2])]:
            continue
        
        #then a mill has been formed!
        return True
    return False
    
def domill(new_index, player, state):
    if stealmill(new_index, player, state): # when a mill has been formed, player can remove opponent's man, but don't remove their own!
        
        if player == player_2: #if human player, select the Man
            R = str(input('You can remove your opponents Man. Which man?:' ))
            while state[board.index(R)] == player:
                print('This is your own Man. Choose an opponent')
                R = str(input('You can remove your opponents Man. Which man?:' ))
            state[board.index(R)] = R
        else: #if AI player, use AlphaBeta Algorithm to select Man

            action = MinimaxDecision_stealingmills(state, cur[new_index], 8)[0] #depth of 8
            state[action] = player #implement the action found by Minimax Decision function
            printGameBoard(state)

            
            
            
def printGameBoard(cur):                       #prints and updates the game board
  # clear screen
  #clear_output()
  # print the board again
  print("Computer Player: Black")
  print("Human Player: White\n")
  for i in range(len(cur)):
      if board[i] == 'A5' or board[i] == 'B4' or board[i] == 'C5' or board[i] == 'D4' or board[i] == 'E5':
        print (cur[i], "\n", end="")
      else: 
        print("", cur[i], "", end="")
      #else:
        #print("\t", end="")

        
game_play()